In [74]:
from torch import empty
import math
import torch

In [75]:
torch.set_grad_enabled(False)

In [55]:
def generate_data(n=1000):
    inputs = empty(n, 2)
    inputs = empty(n, 2).uniform_(0,1)
    labels = (inputs - 0.5).pow(2).sum(1)<1/(2*math.pi)
    return inputs, labels

In [70]:
def split_dataset(inputs, lables, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    train_len = math.floor(inputs.size()[0] * train_ratio)
    val_len = math.floor(inputs.size()[0] * val_ratio)
    test_len = inputs.size()[0]-train_len-val_len
    
    train_inputs = inputs.narrow(0, 0, train_len)
    train_labels = labels.narrow(0, 0, train_len)
    
    validation_inputs = inputs.narrow(0, train_len, val_len)
    validation_labels = labels.narrow(0, train_len, val_len)
    
    test_inputs = inputs.narrow(0, train_len+val_len, test_len)
    test_labels = labels.narrow(0, train_len+val_len, test_len)
    
    return train_inputs, train_labels, validation_inputs, validation_labels, test_inputs, test_labels

In [71]:
inputs, labels = generate_data(n=1000)
train_inputs, train_labels, validation_inputs, validation_labels, test_inputs, test_labels = \
split_dataset(inputs, labels, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2) 


In [83]:
class Module (object) :
    """
    Base class for other neural network modules to inherit from
    """
    
    def __init__(self):
        self._author = 'HB_FB'
    
    def forward ( self , input ) :
        """ `forward` should get for input, and returns, a tensor or a tuple of tensors """
        raise NotImplementedError
        
    def backward ( self , gradwrtoutput ) :
        """
        `backward` should get as input a tensor or a tuple of tensors containing the gradient of the loss 
        with respect to the module’s output, accumulate the gradient wrt the parameters, and return a 
        tensor or a tuple of tensors containing the gradient of the loss wrt the module’s input.
        """
        raise NotImplementedError
        
    def param ( self ) :
        """ 
        `param` should return a list of pairs, each composed of a parameter tensor, and a gradient tensor 
        of same size. This list should be empty for parameterless modules (e.g. activation functions). 
        """
        return []

In [84]:
test = torch.randn(3,5)
test.shape

torch.Size([3, 5])

In [85]:
test.view(-1,1).shape

torch.Size([15, 1])

In [90]:
class ReLu(Module):
    """
    Activation module: ReLu
    
    Outputs:
    forward  :   FloatTensor of size m (m: number of units)
    backward :   FloatTensor of size m (m: number of units)
    """
    def __init__(self):
        super().__init__()
        self.s = 0
        
    def forward(self, input_):
        self.s = inputs
        relu = input_.clamp(min=0)
        return relu
    
    def backward(self, grdwrtoutput):
        relu_input = self.s
        der_relu = relu_input.sign().clamp(min=0)#导数只有可能是1或者0
        gradient_in = grdwrtoutput * der_relu
        return gradient_in    

    def param (self):
        return [(None, None)]  

In [131]:
class Tanh(Module):
    """
    Activation module: Tanh 
    
    Outputs:
    forward  :  FloatTensor of size m (m: number of units)
    backward :  FloatTensor of size m (m: number of units)
    """
    
    def __init__(self):
        super().__init__()
        self.input_ = 0
        
    def forward(self, input_):
        self.input_ = input_
        tanh = 2/(1+(-2*input_).exp())-1
        return tanh
    
    def backward(self, grdwrtoutput):
        result = 4*((self.input_.exp() + (-self.input_).exp()).pow(-2)) * grdwrtoutput
        return result
    
    def param (self):
        return [(None, None)]

In [ ]:
class Sequential(Module):
    """
    A module combining several modules in basic sequential structure
    
    Outputs:
    parameters :  List object containing List objects with the parameters of the modules in the Sequential instance. 
    """
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        args = list(args)[0]
        for ind, module in enumerate(args):
            self.add_module(str(ind), module)

    def add_module(self, ind, module):
        self.ind = module
        self.modules.append(self.ind)
        return module
    
    def forward(self, input):
        out = input
        for module in self.modules:
            out = module.forward(out)
        return out
    
    def backward(self, grdwrtoutput):
        reversed_modules = self.modules[::-1]
        out = grdwrtoutput
        for module in reversed_modules:
            out = module.backward(out)
    
    def param ( self ) :
        parameters = []
        for module in self.modules:
            parameters.append(module.param())
        return parameters